In [75]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

False

In [76]:
# !pip install langchain-community langchain-core

In [77]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [78]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.llm = llm
        self.system_prompt = system_prompt
        # Create a chat prompt that includes 'conversation_history'
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            # Include the conversation history and the last message in the prompt
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

In [79]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY"))

In [80]:
customer_proxy_agent = Agent(
    name="Customer_Proxy",
    system_prompt="""Acts as a representative of the customer. Communicates the customer's high-level requirements to the Product Owner. Communicates the customer's vision to the Product Owner. Communicates the customer's goals to the Product Owner. Provides an overarching view of the products purpose, expected functionalities of the system, and outcomes. 
    Collaborates closely with the Product Owner to guarantee that the product direction remains in line with customer requirements and objectives.""",
    llm=llm
)

In [81]:
product_owner_agent = Agent(
    name="Product_Owner",
    system_prompt="""You are the Product Owner, you are responsable of defining the product's vision, goals, and high-level requirements. Once the customer's problem statement is gathered:
    - First, translate the requirements into high-level product goals. Also show the key performance indicators (KPIs), and crucial user stories.
    - After that, collaborate with Customer Proxy to refine the product vision and ensure alignment with all the customer needs.
    - Once done, work with the Scrum Master to draft a product roadmap, prioritize features, and create the initial product backlog.
    - Once the backlog and initial sprint plan is received from the Scrum Master do the following:
        - Review the product backlog and initial sprint plan based on customer needs and business value.
        - Prepare a comprehensive report for the customer and stakeholders on the product backlog and sprint plan. The Report must include the product vision, goals, high-level requirements, Number of Sprints, Number of Story Points for each sprint, estimated duration of each sprint, description of the sprint and the total project plan.
    - Continuously oversee the product vision to maintain alignment with customer expectations as the development progresses.""",
    llm=llm
)

In [82]:
scrum_master_agent = Agent(
    name="Scrum_Master",
    system_prompt="""You are the Scrum Master, so you have to facilitate the Scrum process, enabling seamless collaboration. During initial planning do the following:
    - Review the product vision, high-level product goals, key performance indicators (KPIs), and essential user stories from the Product Owner.
    - Additionally, Outline key tasks and structure the initial project timeline.
    - You have to communicate everything to the Project Manager, guiding initial sprint planning.
    - Once the specific project plan is received from the Project Manager and use that to create the initial product backlog and sprint plan. Below are descrived all the tasks to be performed:
       - Create a Final Product backlog and divide it into sprints.
       - Each sprint must have a backlog, a goal, estiamtion of story points, and duration.
       - Prepare a report on the product backlog and sprint plan to send it to Product Owner for review. The Report have to include the Number of Sprints, Number of Story Points for each sprint, Backlog and goals for each sprint, estimated duration of each sprint and total project plan.
    """,
    llm=llm
)

In [83]:
project_manager_agent = Agent(
    name="Project_Manager",
    system_prompt="""You are the Project Manager, your primary responsibility is to create the project plan and coordinate it. The project works in an Agile way. During the initial planning phase do the following:
    - You will receive and review the product vision, high-level product goals, key performance indicators (KPIs), and essential user stories, key tasks, and initial project timeline from the Scrum Master.
    - Coordinate with each engineer (Requirements engineer, System engineer, Software engineer, Test engineer, and Documentation engineer) to create the user stories, effort estimates, and specific sprint plan.
    - Once the specific sprint plan is received from each engineer do the following:
      - Complie and review the response from all engineers.
      - Make sure the Scrum Master would be able to calcuate: the backlog, the goal and the estimation of story points for each sprint and duration.
    """,
    llm=llm
)

In [84]:
requirement_engineer_agent = Agent(
    name="Requirement_Engineer",
    system_prompt=("""
    You are the Requirement Engineer, you are responsible for defining detailed user stories and requirements. During the planning phase do the following:
    - Transform the intial sprint plan, high-level customer requirements, key performance indicators (KPIs) into structured user stories based on their requirements, those user stories must have unique identifiers and defined acceptance criteria for each of them.
    - Use previous productivity rates to estimate the effort for each user story, factoring in the complexity of requirements. Take into account we are using a Scrum methodology.
    - Split the user stories into sprints based on the project scope and priorities.
    - Calculate total effort for each sprint, for that you have to take into account the volume of user stories and average productivity, ensuring alignment with overall project scope.
    - Report the detailed user stories and estimated effort to the Project Manager, providing a high-level summary and key requirements for the planning phase.
    """
    ),
    llm=llm
)

In [85]:
system_engineer_agent = Agent(
    name="System_Engineer",
    system_prompt=(
        """You are the System Engineer, you have to design the system architecture and data model based on the requirements provided. During the planning phase do the following:
    - Analyze User Stories so you create a detailed System Design document and Data Model Document.
    - Document each part of the design with unique identifiers, estimating effort using historical productivity data.
    - Calculate total effort using the estimated pages required for design documentation. Take into account we are using a Scrum methodology.
    - Provide the Project Manager with a summary of the system design, including estimated effort for completion, a list of key components, and any identified complexities.
    """
    ),
    llm=llm
)

In [86]:
software_engineer_agent = Agent(
    name="Software_Engineer",
    system_prompt=(
       """You are the Software Engineer, you have to implement features and functionality based on the system design. During the planning phase do the following:
    - Break down the system design into coding tasks, each one need to have an estimated number of lines of code (SLOC).
    - Use previous productivity data to calculate effort requirements for coding tasks, code reviews, and testing. Take into account we are using a Scrum methodology.
    - Provide the Project Manager with a summary of coding tasks, total estimated SLOC, and effort requirements for each major feature.
    - Ensure estimates include effort for Unit Testing, refactoring, and review meetings as needed.
    """
    ),
    llm=llm
)

In [87]:
test_engineer_agent = Agent(
    name="Test_Engineer",
    system_prompt=(
        """You are the Test Engineer, responsible for verifying the product's quality and functionality. During the planning phase do the following:
    - Develop a detailed test plan that aligns with user stories and outlines individual test cases, complete with acceptance criteria.
    - Estimate the effort for creating and executing test cases, defect tracking, and retesting by using historical productivity data. Take into account we are using a Scrum methodology.
    - Provide a structured report to the Project Manager, including the test plan, the estimated number of test cases, and the total effort required for the testing phase.
    - Summarize critical testing activities, such as preparation, defect resolution, and result documentation.
    """
    ),
    llm=llm
)

In [88]:
documentation_engineer_agent = Agent(
    name="Documentation_Engineer",
    system_prompt=(
         """You are the Documentation Engineer, responsible for producing high-quality user and training documentation. During the planning phase:
    - Outline documentation tasks based on user stories and completed product features, including user guides, manuals, and training materials.
    - Collaborate with development and testing teams to gather relevant information and ensure the accuracy of documentation.
    - Estimate the effort for each documentation task using historical productivity data, factoring in time for drafting, review, and revisions. Take into account we are using a Scrum methodology.
    - Provide the Project Manager with a summary report, detailing documentation tasks, the estimated total number of pages, and the effort required.
    - Identify dependencies on development or testing phases and plan incremental updates as new features are completed.
    - Ensure all documentation meets readability and usability standards for the intended audience.
    """
    ),
    llm=llm
)

In [89]:
conversation_prompts = {
    "Project_Manager":(),
    "Requirement_Engineer": (
        "The Scrum Master has provided the initial sprint planning requirements based on the Product Owner's input. "
        "As the Requirements Engineer, your task is to create detailed user stories for requirements gathering and documentation with effort estimates. "
        "Provide a structured report summarizing user stories (give only a summary to other teams but provide a complete list to the Project Manager), "
        "effort estimates, and effort calculations for the requirement engineering phase to support sprint planning. "
        "At the end, summarize the total number of user stories, the total number of requirements, and the total effort for the requirement engineering phase."
        
    "The provided historical data for the requirement engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the requirement engineering phase:"
    "Here is the amount of work and productivity rate for the requirement engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           112 Req                 5 Req/Hour"
        "Review Requirments                                                            "
        "       Preparation for Review                112  Req               18 Req/Hour"
        "       Review Meeting                        112 Req                28 Req/Hour"
        "Rework                                       136 defects            10 defects/Hour"

        "Project 2"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           389 Req                 3 Req/Hour"
        "Review Requirments                                                      "
        "       Preparation for Review                389  Req               5 Req/Hour"
        "       Review Meeting                        389 Req                8 Req/Hour"
        "Rework                                       413 defects            5 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           210 Req                 4 Req/Hour"
        "Review Requirments                                                      "
        "       Preparation for Review                210  Req               12 Req/Hour"
        "       Review Meeting                        210 Req                18 Req/Hour"
        "Rework                                       239 defects            8 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "System_Engineer": (
        "The Requirements Engineer has completed the initial user stories. Based on these, draft user stories for system design and data model creation, including effort estimates for each task. "
    "Reference historical data for accuracy and provide a structured report with summarized user stories (give only a summary to other teams but provide a complete list to the Project Manager), "
    "effort estimates, and effort calculations for the system engineering phase to support sprint planning. "
    "At the end, summarize the total number of user stories, total requirements, and the total effort for the system engineering phase."

    "The provided historical data for the system engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the system engineering phase:"    
    "Here is the amount of work and productivity rate for the system engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Analysis Document                      66 Pages                 5 Pages/Hour"
        "Review AD                                                            "
        "       Preparation for AD                 66 Pages                    4 Pages/Hour"
        "       Review Meeting                        66 Pages                9 Pages/Hour"
        "Rework                                       88 defects            6 defects/Hour"
        "Write Design Document                       123 defects                 6 Page/Hour"
        "Review DD                                                            "
        "       Preparation for DD                 89 Pages              4 Req/Hour"
        "       Review Meeting                        89 Pages                8 Pages/Hour"
        "Rework                                       188 defects            5 defects/Hour"
        "Write Data Model (DM)                        28 Pages               1page/4hours"
        "Review DM                                                                     "
        "       Preparation for DM                 28 Pages                   4 page/hour"
        "       Review Meeting                        28 Pages                5 page/hour"
        "Rework                                       88 defects            6 defects/Hour"

        "Project 2"
        "Task                                                          Amount of Work      Productivity Rate"
        "Write Analysis/Design Document                                     234 Pages                 4 Pages/Hour"
        "Review Analysis/Design Document                                                           "
        "       Preparation for Analysis/Design Document                   234 Pages              5 Pages/Hour"
        "       Review Meeting                                             234 Pages               8 Pages/Hour"
        "Rework                                                             509 defects            7 defects/Hour"
        "Write Data Model (DM)                                             62 Pages               1 page/ hour"
        "Review DM                                                                     "
        "       Preparation for DM                    62 Pages                 4 page/hour"
        "       Review Meeting                        62 Pages                 10 page/hour"
        "Rework                                       378 defects            6 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Analysis Document                      172 Pages                 5 Pages/Hour"
        "Review AD                                                            "
        "       Preparation for AD                   172 Pages                    5 Pages/Hour"
        "       Review Meeting                        172 Pages                9 Pages/Hour"
        "Rework                                       347 defects            7 defects/Hour"
        "Write Design Document                       220 Pages                 5 Pages/Hour"
        "Review DD                                                            "
        "       Preparation for DD                    220 Pages              5 Pages/Hour"
        "       Review Meeting                        220 Pages               8 Pages/Hour"
        "Rework                                       472 defects            6 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Software_Engineer": (
        "The System Engineer has finalized the design phase user stories. Your task as the Software Engineer is to create detailed user stories for coding tasks, including effort estimates for each. "
    "Provide a structured report summarizing user stories (give only a summary to other teams but provide the complete list to the Project Manager), along with effort estimates and effort calculations for the software engineering phase to support sprint planning. "
    "At the end, summarize the total number of user stories, the total number of requirements, and the total effort for the software engineering phase."

    "The provided historical data for the software engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the software engineering phase:"
    "Here is the amount of work and productivity rate for the software engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Code                                  3800 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            256 test cases           10 Test Cases/Day"
        "       Fix Found Defects                     179 Defects              8 Defects/Day"
        "       Test Fixed Defects                    179 Defects              12 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       3800 SLOC                 113 SLOC/Hour"
        "       Code Inspection Meeting               3800 SLOC                 189 SLOC/Hour"
        "Rework                                       188 defects                5 defects/Hour"

        "Project 2"
        "Task                                    Amount of Work           Productivity Rate"
        "Write Code                                  9725 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            1045 test cases           5 Test Cases/Day"
        "       Fix Found Defects                     902 Defects              12 Defects/Day"
        "       Test Fixed Defects                    902 Defects              8 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       9725 SLOC                 90 SLOC/Hour"
        "       Code Inspection Meeting               9725 SLOC                 135 SLOC/Hour"
        "Rework                                       1230 defects                5 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work           Productivity Rate"
        "Write Code                                  3800 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            332 test cases           25 Test Cases/Day"
        "       Fix Found Defects                     266 Defects              10 Defects/Day"
        "       Test Fixed Defects                    266 Defects              10 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       3800 SLOC                 102 SLOC/Hour"
        "       Code Inspection Meeting               3800 SLOC                 102 SLOC/Hour"
        "Rework                                       334 defects                5 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Test_Engineer": (
        "The Software Engineer has outlined user stories for coding tasks. Based on this, please prepare user stories for test planning and execution. "
    "Estimateeffortfor writing test cases, execution, and rework based on historical test data. Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the software engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the test engineering phase."

    "The provided historical data for the test engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the test engineering phase:"
    "Here is the amount of work and productivity rate for the test engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    166 pages            5 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            166 Pages           4 Pages/Hour"
        "       Review Meeting                    166 Pages            8 Pages/Hour"
        "Rework                                   255 defects          5 defects/Hour"
        "Execute Test Plan                        215 Test Cases     9Test Cases/day"
        "Fix Found Defects                        213 Defects          14 Defects/day"
        "Test Fixed Defects                       213 Defects          17 Defects/day"

        "Project 2"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    417 pages            8 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            417 Pages           5 Pages/Hour"
        "       Review Meeting                    417 Pages            10 Pages/Hour"
        "Rework                                   502 defects          5 defects/Hour"
        "Execute Test Plan                        1167 Test Cases      8 Test Cases/day"
        "Fix Found Defects                        645 Defects          5 Defects/day"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    205 pages            8 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            205 Pages           5 Pages/Hour"
        "       Review Meeting                    205 Pages            10 Pages/Hour"
        "Rework                                   101 defects          4 defects/Hour"
        "Execute Test Plan                        194 Test Cases      8 Test Cases/day"
        "Fix Found Defects                        113 Defects          5 Defects/day"
        "Test Fixed Defect                        113 Defects          10 Defects/day"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Documentation_Engineer": (
        "The Test Engineer has completed user stories for testing. Please create user stories for documentation tasks, "
    "Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the software engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the documentation engineering phase."

    "The provided historical data for the documentation engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the documentation engineering phase:"
    "Here is the amount of work and productivity rate for the documentation engineering phase for the previous projects:"

    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                     134 Pages                 4 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            134 Pages               5 Pages/Hour"
    "       Review Meeting                    134 Pages                7 Pages/Hour"
    "Rework                                   174 defects            6 defects/Hour"

    "Project 2"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      389 Pages                 6 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            389 Pages                5 Pages/Hour"
    "       Review Meeting                    389 Pages                7 Pages/Hour"
    "Rework                                   532 defects            4 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      177 Pages                 5 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            177 Pages                5 Pages/Hour"
    "       Review Meeting                    177 Pages                7 Pages/Hour"
    "Rework                                   236 defects            5 defects/Hour"

    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Scrum_Master": (
       "The outputs from all engineers have been received. I have compiled a complete backlog with user stories, time, and work estimates "
    "for each phase. This backlog is now ready for review and further structuring for the initial Scrum planning. "
    "Please find the backlog summary with user stories, effort estimates, and phase-specific effort estimates in the report attached. PLea review it and generate the final backlog and draft an approximate number of sprints required to complete the project and the user stories to be accomplished in each sprint."
    "Once done, report back the summary of the total number of sprints, total number of user stories and the total effort for the project."
    ),
    "Product_Owner": (
        "The Project Manager has compiled a complete backlog, which I have reviewed and structured for the initial Scrum plan. "
    "Based on the total user stories and effort estimates, I have drafted an approximate number of sprints required to complete the project. "
    "The backlog items have been divided into sprints, with rough sprint goals set for each. Please review this structured plan as we prepare for sprint kickoff."
    "Once you have finalised and prioritised the user stories for each sprint, report back the total number of sprints, total number of user stories and the total effort for the project."
    ),
    "Customer_Proxy":("The Scrum Master has provided an initial Scrum plan. I have reviewed and prioritized the user stories in the backlog, adjusting the sprint "
    "plan as necessary to align with our product goals. The final sprint breakdown, including prioritized user stories and sprint objectives, is now ready. "
    "This report includes the sprint goals, total number of sprints, and estimated completion timeline for review by the Customer Proxy.")
}

In [90]:
# Initialize the shared conversation history
conversation_history = ""

# Initial message from Customer Proxy
initial_message = (
    "I want to create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis. "
    "The system should process high-resolution images, provide real-time diagnostics, and support doctors with detailed reports. "
    "This project should prioritize accuracy, speed, and usability for healthcare providers."
    "Please document the initial product vision, objectives, and KPIs, including primary user stories that will guide development and outline a high-level product backlog focused on prioritized features and metrics to assess product value. "
)

# Add the initial message to the conversation history
conversation_history += f"Customer_Proxy: {initial_message}\n"
message_count = 1


# Customer Proxy sends message to Product Owner
product_owner_response = product_owner_agent.respond(initial_message, conversation_history=conversation_history)
print(f"{product_owner_agent.name}:\n{product_owner_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Customer Proxy and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Customer Proxy responds
    customer_proxy_response = customer_proxy_agent.respond(product_owner_response, conversation_history=conversation_history)
    print(f"{customer_proxy_agent.name}:\n{customer_proxy_response}\n")
    user_input = input("Press 'Enter' to continue the conversation between Customer Proxy and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Project Manager responds
    product_owner_response = product_owner_agent.respond(customer_proxy_response, conversation_history=conversation_history)
    print(f"{product_owner_agent.name}:\n{product_owner_response}\n")



second_message = (
    "I have gathered the requirements from the Customer Proxy."
    "Create an initial plan for sprint planning, including essential roles, estimated timeframes, and key milestones. Note that you do not have to estimate the number of sprints and backlog items for each sprint unles your get all the project plan back from project manager. "
    "Please perform the initial sprint planning with the development team, ensuring each phase is planned around the structured backlog."
)

conversation_history += f"Product_Owner: {second_message}\n"


# Product Owner sends message to Scrum Master
scrum_master_response= scrum_master_agent.respond(second_message, conversation_history=conversation_history)
print(f"{scrum_master_agent.name}:\n{scrum_master_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    #  Product Owner responds
    product_owner_response = product_owner_agent.respond(scrum_master_response, conversation_history=conversation_history)
    print(f"{product_owner_agent.name}:\n{product_owner_response}\n")

    
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Scrum Master responds
    scrum_master_response = scrum_master_agent.respond(product_owner_response, conversation_history=conversation_history)
    print(f"{scrum_master_agent.name}:\n{scrum_master_response}\n")
    

third_message = (

    "The Product Owner has provided the project vision and initial backlog for the skin anomaly detection system." 
    "I have outlined a preliminary plan for sprint planning, including essential roles, estimated timeframes, and key milestones." 
    "As the Project Manager, please create a comprehensive project plan with detailed user stories and effort estimates for each. Ensure that the plan is finalized only after receiving input from the development team." 
    "Our goal is to have a well-organized backlog based on user stories, with clear effort estimates for each." 
    "Please collaborate with each engineer to gather relevant use cases and provide accurate effort estimates."
)

conversation_history += f"Scrum_Master: {third_message}\n"
last_message = third_message

# Scrum Master sends message to Project Manager 
project_manager_response = project_manager_agent.respond(third_message, conversation_history=conversation_history)
print(f"{project_manager_agent.name}:\n{project_manager_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Project Manager, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Scrum Master responds
    scrum_master_response = scrum_master_agent.respond(project_manager_response, conversation_history=conversation_history)
    print(f"{scrum_master_agent.name}:\n{scrum_master_response}\n")
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Project Manager, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Project Manager responds
    project_manager_response = project_manager_agent.respond(scrum_master_response, conversation_history=conversation_history)
    print(f"{project_manager_agent.name}:\n{project_manager_response}\n")






Product_Owner:
### Initial Product Vision

**Vision Statement:**
To develop an AI-powered computer vision system that accurately detects skin anomalies, such as cancer and psoriasis, from high-resolution images, providing healthcare providers with real-time diagnostics and comprehensive reports to enhance patient care.

---

### Product Objectives

1. **Accuracy:** Achieve a diagnostic accuracy rate of at least 95% for skin anomaly detection to ensure reliable support for healthcare professionals.
2. **Speed:** Enable real-time image processing and diagnostics within 2 minutes to facilitate timely decision-making in clinical settings.
3. **Usability:** Create an intuitive user interface that allows healthcare providers to easily upload images, receive diagnostics, and generate reports.
4. **Integration:** Ensure seamless integration with existing healthcare systems (e.g., Electronic Health Records) to streamline workflows for healthcare providers.
5. **Scalability:** Design the system 

In [91]:
agents = [
    project_manager_agent,
    requirement_engineer_agent,
    system_engineer_agent,
    software_engineer_agent,
    test_engineer_agent,
    documentation_engineer_agent,
    scrum_master_agent,
    product_owner_agent,
    customer_proxy_agent
]
# Set the maximum number of messages and the message count
max_messages = 20
exit_loop = False  # Flag to control breaking out of both loops

# Start the conversation loop
while message_count < max_messages and not exit_loop:
    for agent in agents:
        user_input = input("Press 'Enter' to continue the conversation between the group, or type 'exit' to end it: ")
        if user_input.lower() == 'exit':
            exit_loop = True
            break  # Breaks out of the inner loop

        # Project Manager sends the specific prompt to the agent
        prompt = conversation_prompts[agent.name]
        # Project Manager sends the prompt as a message
        pm_message = prompt
        conversation_history += f"{project_manager_agent.name}: {pm_message}\n"

        # Agent responds to the last message with access to the conversation history
        agent_response = agent.respond(
            input_text=last_message,
            conversation_history=conversation_history
        )

        # Print the agent's response
        print(f"{agent.name}:\n{agent_response}\n")

        # Update the conversation history
        conversation_history += f"{agent.name}: {agent_response}\n"
        # Update the last message
        last_message = agent_response
        message_count += 1

        if message_count >= max_messages or exit_loop:
            break  # Breaks out of the inner loop if conditions are met

# Exit the program after breaking out of both loops
print("Conversation ended.")

Project_Manager:
### Initial Project Plan for AI-Powered Skin Anomaly Detection System

#### Product Vision
To create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis, by processing high-resolution images, providing real-time diagnostics, and supporting healthcare providers with detailed reports.

#### Objectives
1. **Accuracy**: Achieve a diagnostic accuracy rate of over 95%.
2. **Speed**: Provide real-time analysis within 5 seconds of image upload.
3. **Usability**: Ensure the system is user-friendly and accessible for healthcare providers, with an intuitive interface.

#### Key Performance Indicators (KPIs)
- **Diagnostic Accuracy**: Percentage of correct diagnoses made by the system.
- **Response Time**: Average time taken for the system to analyze and provide results.
- **User Satisfaction**: Feedback from healthcare providers on usability and effectiveness.
- **System Uptime**: Percentage of time the system is operational and a